## Un esempio di esecuzione automatica di funzioni ("_tool_") da un modello linguistico locale

Luca Mari, ottobre 2024  

Quest'opera è distribuita con <a href="http://creativecommons.org/licenses/by-nc-sa/4.0" target="_blank">Licenza Creative Commons Attribuzione - Non commerciale - Condividi allo stesso modo 4.0 Internazionale</a>.  
<img src="https://creativecommons.it/chapterIT/wp-content/uploads/2021/01/by-nc-sa.eu_.png" width="100">

**Obiettivo**: comprendere qualche aspetto della logica delle architetture ad agenti e dell'esecuzione automatica di funzioni.  
**Precompetenze**: basi di Python.

> Per eseguire questo notebook, supponiamo con VSCode, occorre:
> * installare un interprete Python
> * scaricare da https://ollama.com e installare Ollama
> * scaricare da Ollama un modello capace di operare con strumenti, supporremo `llama3.1:8b`:  
>       `ollama pull llama3.1`
> * scaricare da https://code.visualstudio.com/download e installare VSCode
> * eseguire VSCode e attivare le estensioni per Python e Jupyter
> * ancora in VSCode:
>     * creare una cartella di lavoro e renderla la cartella corrente
>     * copiare nella cartella il file di questa attività: [websearchingagents.ipynb](websearchingagents.ipynb)
>     * aprire il notebook `websearchingagents.ipynb`
>     * creare un ambiente virtuale locale Python (Select Kernel | Python Environments | Create Python Environment | Venv, e scegliere un interprete Python):
>     * installare i moduli Python richiesti, eseguendo dal terminale:  
>         `pip install pyautogen duckduckgo_search`
> * eseguire dalla linea di comando:  
>       `OLLAMA_MODELS=xxx OLLAMA_HOST=127.0.0.1:1234 ollama serve`  
> dove `xxx` è la directory che contiene i modelli Ollama (in Linux potrebbe essere `/var/lib/ollama/.ollama/models`)

Importiamo i moduli Python necessari e specifichiamo la configurazione per il modello linguistico che sarà usato (deve essere in grado di operare con strumenti) e l'indirizzo del server su cui sarà in esecuzione in locale (a sua volta, il server deve essere in grado di gestire strumenti: al momento Ollama, ma non LM Studio).

In [3]:
import autogen
from typing_extensions import Annotated
from duckduckgo_search import DDGS

llm_config = {
    "config_list": [{ "base_url":"http://localhost:1234/v1",
                      "model":"llama3.1:8b",
                      "api_key":"not_used" }],
    "timeout": 120,
    "cache_seed": None,
}

Definiamo una semplice architettura con un agente di interfaccia ed esecutore (`user_proxy`) e un agente che gestisce il modello linguistico (`domain_expert`).

In [13]:
user_proxy = autogen.UserProxyAgent(
    name="interfaccia con l'utente ed esecutore di codice",
    is_termination_msg=(lambda msg: "conclus" in msg["content"].lower()), # a volte potrebbe essere "concluso" o "conclusa"...
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
    max_consecutive_auto_reply=1,
)

domain_expert = autogen.AssistantAgent(
    name="esperto di dominio",
    system_message="Se ti sono richieste informazioni su libri, usa solo la funzione disponibile per la ricerca nel tuo archivio interno, senza mai ricorrere alla tua memoria. Quando hai completato la ricerca, scrivi CONCLUSO.",
    llm_config=llm_config,
)

Questa è la funzione Python che dovrebbe essere eseguita quando richiesto, resa disponibile all'agente `esperto di dominio` grazie ai decoratori (per semplicità manteniamo il contenuto dell'archivio su cui fare ricerche direttamente all'interno della funzione).

In [14]:
@user_proxy.register_for_execution()
@domain_expert.register_for_llm(description="Cerca informazioni nel web.")
def cerca(
        query: Annotated[str, "condizione di ricerca"]
    ) -> str:
    return DDGS().text(query, max_results=5)

Verifichiamo che, grazie ai decoratori, l'agente `esperto di dominio` sia stato dotato dello schema json con la dichiarazione della funzione (la documentazione dei "tools" per gli "Assistants" è qui:  
https://platform.openai.com/docs/guides/function-calling  
https://platform.openai.com/docs/api-reference/assistants/modifyAssistant#assistants-modifyassistant-tools).

In [15]:
domain_expert.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Cerca informazioni nel web.',
   'name': 'cerca',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'condizione di ricerca'}},
    'required': ['query']}}}]

Ecco dunque un esempio di uso di questa architettura.

In [16]:
domain_expert.reset()
res = user_proxy.initiate_chat(domain_expert, message="Com'è definita l'unità di misura Joule?")

interfaccia con l'utente ed esecutore di codice (to esperto di dominio):

Com'è definita l'unità di misura Joule?

--------------------------------------------------------------------------------
Com'è definita l'unità di misura Joule?

--------------------------------------------------------------------------------
[autogen.oai.client: 10-04 14:49:05] {349} WARNING - Model llama3.1:8b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
esperto di dominio (to interfaccia con l'utente ed esecutore di codice):


***** Suggested tool call (call_a8xbwhyu): cerca *****
Arguments: 
{"query":"definizione joule"}
******************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION cerca...
interfaccia con l'utente ed esecutore di codice (to esperto di dominio):

interfaccia con l'utente ed esecutor

E questa è la ricostruzione della conversazione tra i due agenti.

In [17]:
from pprint import pprint
pprint(res.chat_history)

[{'content': "Com'è definita l'unità di misura Joule?",
  'name': "interfaccia con l'utente ed esecutore di codice",
  'role': 'assistant'},
 {'content': '',
  'role': 'assistant',
  'tool_calls': [{'function': {'arguments': '{"query":"definizione joule"}',
                               'name': 'cerca'},
                  'id': 'call_a8xbwhyu',
                  'type': 'function'}]},
 {'content': '[{"title": "Joule - Wikipedia", "href": '
             '"https://it.wikipedia.org/wiki/Joule", "body": "Un joule può '
             'essere definito come il lavoro svolto esercitando la forza di un '
             'newton per una distanza di un metro, perciò la stessa quantità '
             'può essere riferita come newton metro. Comunque, per evitare '
             'confusione, il newton metro è tipicamente usato come la misura '
             'del momento meccanico e non dell\'energia."}, {"title": "Joule - '
             'Wikipedia", "href": "https://en.wikipedia.org/wiki/Joule", '
      

È un semplice ma già interessante esempio (modello linguistico di piccole dimensioni, esecuzione locale, in italiano...) di un mix tra un "Sistema 1" (il modello linguistico stesso) e un "Sistema 2" (la funzione Python).